In [5]:
import os
import sqlite3
import numpy as np
import pandas as pd

from pmdarima.arima import auto_arima

from sklearn.metrics import mean_absolute_percentage_error as mape

import pickle

os.chdir(os.path.dirname(os.path.abspath('__file__')))

In [40]:
def train_with_all_data(filename):
    # cargar toda la data de la db
    conn = sqlite3.connect('DB/daily_visits.db')
    df = pd.read_sql("SELECT * FROM visit_quant", conn, parse_dates='Date', index_col='Date')
    conn.close()

    # aplicar log y quitar los -inf para evitar errores
    df_logScale = np.log(df)
    df_logScale[df_logScale.index.isin(np.isfinite(df_logScale[['Users']]).query('not Users').index)] = 0

    # entrenar modelo
    users_log = df_logScale['Users'].values
    model_new = auto_arima(users_log,
                           start_p = 1,
                           start_q = 1,
                           max_p = 10,
                           max_q = 10,
                           d = 1)
    model_new.fit(users_log)
        
    with open(filename, "wb") as output_file:
        pickle.dump(model_new, output_file)

In [ ]:
def update_model(filename):
    # cargar modelo
    with open(filename, 'rb') as archivo_entrada:
        model = pickle.load(archivo_entrada)
    
    #cargar data nueva
    conn = sqlite3.connect('DB/daily_visits.db')
    cursor = conn.cursor()
    cursor.execute("SELECT Date FROM visit_quant WHERE Date=(SELECT max(Date) FROM visit_quant)")
    last_trained_date = cursor.fetchall()
    
    cursor.close()
    conn.close()
    

In [36]:
filename = 'model_arima_users.pkl'
if os.path.isfile(filename):
    update_model(filename)
else:
    train_with_all_data(filename)
        


c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\internals\blocks.py:329: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


In [29]:
with open('src/model_arima_users.pkl', 'rb') as archivo_entrada:
    model = pickle.load(archivo_entrada)
model

ARIMA(order=(6, 1, 3), scoring_args={}, suppress_warnings=True,
      with_intercept=False)

In [30]:
len(model.fittedvalues())

851

In [28]:
model.params()

array([ 0.00620121, -0.1445652 , -0.69013257, -0.43413921, -0.13465044,
       -0.14464045, -0.47703756,  0.07183452,  0.69338446,  0.2252379 ])

In [31]:
model.params()

array([ 0.00675093, -0.14540411, -0.68892706, -0.43427878, -0.13444745,
       -0.14352465, -0.47690728,  0.0713751 ,  0.69378339,  0.21902508])

In [34]:
np.exp(model.predict(9))

array([28.21708512, 36.82631348, 40.08357849, 40.40306004, 32.60634495,
       27.5081774 , 25.12930063, 28.29796715, 34.94426183])

In [13]:
conn = sqlite3.connect('DB/daily_visits.db')
cursor = conn.cursor()
cursor.execute("SELECT Date FROM visit_quant WHERE Date=(SELECT max(Date) FROM visit_quant)")
last_trained_date = cursor.fetchall()
# df_new = pd.read_sql("SELECT * FROM visit_quant", conn, parse_dates='Date', index_col='Date')
print(last_trained_date)
cursor.close()
conn.close()
# df_new

[('2022-04-30 00:00:00',)]


In [61]:
df = pd.read_csv('data/users_web.csv')
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
df

,Date,Users
0,2020-01-01,13
1,2020-01-02,17
2,2020-01-03,13
3,2020-01-04,4
4,2020-01-05,10
...,...,...
846,2022-04-26,63
847,2022-04-27,64
848,2022-04-28,46
849,2022-04-29,29


In [5]:
os.getcwd()

'c:\\Users\\Usuario\\Desktop\\ML-Visitas'

In [12]:
os.chdir('src')
a = '../data/'
b = 'users_web.csv'
os.path.isfile(a+b)

True